#### This file demonstrates the mode selection process, and is used to populate table X in paper. 

In [ ]:
import numpy as np
import json
import bgp_qnm_fits as bgp
import time

SIM_ID = "0001"
DATA_TYPE = "news"
N_MAX = 6
L_MAX = 7

SPHERICAL_MODES_PES = [(2, 2), (3, 2), (4, 2)]

INITIAL_MODES = []  # [(*s, 0, 1 if s[1] > 0 else -1) for s in SPHERICAL_MODES_PES]

CANDIDATE_MODES = [
    (*s, n, 1) for s in SPHERICAL_MODES_PES for n in range(0, N_MAX + 1)
] + [(*s, n, -1) for s in SPHERICAL_MODES_PES for n in range(0, N_MAX + 1)]

THRESHOLD = 0.9999
T0 = 20
T = 100
N_DRAWS = 1000
INCLUDE_CHIF = False
INCLUDE_MF = False

log_threshold = np.log(THRESHOLD)

In [3]:
tuned_param_dict_GP = bgp.get_tuned_param_dict("GP", data_type=DATA_TYPE)[SIM_ID]
sim = bgp.SXS_CCE(SIM_ID, type=DATA_TYPE, lev="Lev5", radius="R2")

/data/vault/rvnd2/Code/qnmfits/qnmfits/Waveforms/Base.py:116: RuntimeWarning: divide by zero encountered in divide
  chidot = self.Jdot/(self.Moft**2).reshape(len(self.Moft),1)


In [ ]:
select_object = bgp.BGP_select(
    sim.times,
    sim.h,
    INITIAL_MODES,
    sim.Mf,
    sim.chif_mag,
    tuned_param_dict_GP,
    bgp.kernel_GP,
    t0=T0,
    candidate_modes=CANDIDATE_MODES,
    log_threshold=log_threshold,
    candidate_type="all",
    num_draws=N_DRAWS,
    T=T,
    spherical_modes=SPHERICAL_MODES_PES,
    include_chif=INCLUDE_CHIF,
    include_Mf=INCLUDE_MF,
    data_type=DATA_TYPE,
)

In [ ]:
np.exp(-0.0001921868425761782)

0.9998078316241321

In [ ]:
modes_lists = select_object.full_modes
dot_products = select_object.dot_products

In [ ]:
mismatch_22 = []
mismatch_32 = []
mismatch_44 = []

for i, mode in enumerate(modes_lists):

    model = modes_lists[: i + 1]

    fit = bgp.BGP_fit(
        sim.times,
        sim.h,
        model,
        sim.Mf,
        sim.chif_mag,
        tuned_param_dict_GP,
        bgp.kernel_GP,
        t0=T0,
        T=T,
        decay_corrected=True,
        spherical_modes=SPHERICAL_MODES_PE,
        include_chif=INCLUDE_CHIF,
        include_Mf=INCLUDE_MF,
        data_type=DATA_TYPE,
    )

    mismatch_22.append(
        bgp.mismatch(
            fit.fit["model_array_linear"][0:1],
            fit.fit["data_array_masked"][0:1],
            fit.fit["noise_covariance"][0:1],
        )
    )
    mismatch_32.append(
        bgp.mismatch(
            fit.fit["model_array_linear"][1:2],
            fit.fit["data_array_masked"][1:2],
            fit.fit["noise_covariance"][1:2],
        )
    )
    mismatch_44.append(
        bgp.mismatch(
            fit.fit["model_array_linear"][2:3],
            fit.fit["data_array_masked"][2:3],
            fit.fit["noise_covariance"][2:3],
        )
    )

In [ ]:
import pandas as pd

# Prepare table data
table_data = []
for i, (mode, dot, m22, m32, m44) in enumerate(
    zip(modes_lists, dot_products, mismatch_22, mismatch_32, mismatch_44), 1
):
    sign = "+" if mode[3] == 1 else "-"
    if len(mode) == 8:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})^2"
    else:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})"
    table_data.append(
        {
            "Iteration": i,
            "Mode Added": mode_name,
            "Dot Product": dot,
            "Mismatch 22": m22,
            "Mismatch 32": m32,
            "Mismatch 44": m44,
        }
    )

# Pandas styled table
df = pd.DataFrame(table_data)
display(
    df.style.format(
        {
            "Dot Product": "{:.3g}",
            "Mismatch 22": "{:.3g}",
            "Mismatch 32": "{:.3g}",
            "Mismatch 44": "{:.3g}",
        }
    )
)

,Iteration,Mode Added,Dot Product,Mismatch 22,Mismatch 32,Mismatch 44
0,1,"(2, 2, 0, +)",1.41e+07,0.0568,0.749,nan
1,2,"(3, 2, 0, +)",2.75e+06,0.0565,0.0999,nan
2,3,"(2, 2, 1, +)",1.8e+06,0.00202,0.0781,nan
3,4,"(3, 2, 1, +)",5.28e+05,0.0018,0.004,nan
4,5,"(2, 2, 2, +)",5.96e+04,7.8e-05,0.0032,nan
5,6,"(3, 2, 2, +)",2.11e+04,6.42e-05,0.000375,nan
6,7,"(2, 2, 3, +)",1.93e+03,6.08e-06,0.000359,nan
7,8,"(3, 2, 3, +)",674,5.58e-06,0.00027,nan
8,9,"(2, 2, 4, +)",104,1.81e-06,0.000272,nan
9,10,"(3, 2, 4, +)",120,1.84e-06,0.000255,nan


In [ ]:
import numpy as np

# Plain text for LaTeX with Significance column, three significant figures, scientific notation
print("%")
for row in table_data:
    dot_product = f"{row['Dot Product']/2:.3g}"
    m22 = f"{float(row['Mismatch 22']):.3g}"
    m32 = f"{float(row['Mismatch 32']):.3g}"
    m44 = f"{float(row['Mismatch 44']):.3g}"
    dot_product_sci = f"{float(dot_product):.2e}"
    base, exp = dot_product_sci.split("e")
    dot_product_tex = f"{base}\\times 10^{{{int(exp)}}}"
    significance = (
        f"1 - \exp(-{dot_product_tex}) \\approx {1 - np.exp(-float(dot_product)):.3g}"
    )
    print(
        f"{row['Iteration']} & ${row['Mode Added']}$ & ${significance}$ & {m22} & {m32} & {m44} \\\ %"
    )

%
1 & $(2, 2, 0, +)$ & $1 - \exp(-7.07\times 10^{6}) \approx 1$ & 0.0568 & 0.749 & nan \\ %
2 & $(3, 2, 0, +)$ & $1 - \exp(-1.38\times 10^{6}) \approx 1$ & 0.0565 & 0.0999 & nan \\ %
3 & $(2, 2, 1, +)$ & $1 - \exp(-9.01\times 10^{5}) \approx 1$ & 0.00202 & 0.0781 & nan \\ %
4 & $(3, 2, 1, +)$ & $1 - \exp(-2.64\times 10^{5}) \approx 1$ & 0.0018 & 0.004 & nan \\ %
5 & $(2, 2, 2, +)$ & $1 - \exp(-2.98\times 10^{4}) \approx 1$ & 7.8e-05 & 0.0032 & nan \\ %
6 & $(3, 2, 2, +)$ & $1 - \exp(-1.06\times 10^{4}) \approx 1$ & 6.42e-05 & 0.000375 & nan \\ %
7 & $(2, 2, 3, +)$ & $1 - \exp(-9.67\times 10^{2}) \approx 1$ & 6.08e-06 & 0.000359 & nan \\ %
8 & $(3, 2, 3, +)$ & $1 - \exp(-3.37\times 10^{2}) \approx 1$ & 5.58e-06 & 0.00027 & nan \\ %
9 & $(2, 2, 4, +)$ & $1 - \exp(-5.20\times 10^{1}) \approx 1$ & 1.81e-06 & 0.000272 & nan \\ %
10 & $(3, 2, 4, +)$ & $1 - \exp(-6.02\times 10^{1}) \approx 1$ & 1.84e-06 & 0.000255 & nan \\ %
11 & $(3, 2, 5, +)$ & $1 - \exp(-4.04\times 10^{1}) \approx 1$ & 2.0